In [ ]:
import pandas as pd

In [ ]:
index_kurento_names = [
    "loadtest-webrtc-final-kurento-2p-t3medium",
    "loadtest-webrtc-final-kurento-2p-t3medium-2",
    "loadtest-webrtc-final-kurento-2p-t3medium-wait",
    "loadtest-webrtc-final-kurento-2p-t3medium-retry",
    "loadtest-webrtc-final-kurento-5p-t3medium",
    "loadtest-webrtc-final-kurento-5p-t3medium-2",
    "loadtest-webrtc-final-kurento-5p-t3medium-wait",
    "loadtest-webrtc-final-kurento-5p-t3medium-retry",
    "loadtest-webrtc-final-kurento-8p-t3medium",
    "loadtest-webrtc-final-kurento-8p-t3medium-2",
    "loadtest-webrtc-final-kurento-8p-t3medium-wait",
    "loadtest-webrtc-final-kurento-8p-t3medium-retry",
    "loadtest-webrtc-final-kurento-3p-10s-t3medium",
    "loadtest-webrtc-final-kurento-3p-10s-t3medium-2",
    "loadtest-webrtc-final-kurento-3p-10s-t3medium-wait",
    "loadtest-webrtc-final-kurento-3p-10s-t3medium-retry",
    "loadtest-webrtc-final-kurento-3p-20s-t3medium",
    "loadtest-webrtc-final-kurento-3p-20s-t3medium-2",
    "loadtest-webrtc-final-kurento-3p-20s-t3medium-wait",
    "loadtest-webrtc-final-kurento-3p-20s-t3medium-retry",
    "loadtest-webrtc-final-kurento-3p-40s-t3medium",
    "loadtest-webrtc-final-kurento-3p-40s-t3medium-2",
    "loadtest-webrtc-final-kurento-3p-40s-t3medium-wait",
    "loadtest-webrtc-final-kurento-3p-40s-t3medium-retry",
]

index_mediasoup_names = [
    "loadtest-webrtc-final-mediasoup-2p-t3medium",
    "loadtest-webrtc-final-mediasoup-2p-t3medium-2",
    "loadtest-webrtc-final-mediasoup-2p-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-2p-t3medium-retry",
    "loadtest-webrtc-final-mediasoup-5p-t3medium",
    "loadtest-webrtc-final-mediasoup-5p-t3medium-2",
    "loadtest-webrtc-final-mediasoup-5p-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-5p-t3medium-retry",
    "loadtest-webrtc-final-mediasoup-8p-t3medium",
    "loadtest-webrtc-final-mediasoup-8p-t3medium-2",
    "loadtest-webrtc-final-mediasoup-8p-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-8p-t3medium-retry",
    "loadtest-webrtc-final-mediasoup-3p-10s-t3medium",
    "loadtest-webrtc-final-mediasoup-3p-10s-t3medium-2",
    "loadtest-webrtc-final-mediasoup-3p-10s-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-3p-10s-t3medium-retry",
    "loadtest-webrtc-final-mediasoup-3p-20s-t3medium",
    "loadtest-webrtc-final-mediasoup-3p-20s-t3medium-2",
    "loadtest-webrtc-final-mediasoup-3p-20s-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-3p-20s-t3medium-retry",
    "loadtest-webrtc-final-mediasoup-3p-40s-t3medium",
    "loadtest-webrtc-final-mediasoup-3p-40s-t3medium-2",
    "loadtest-webrtc-final-mediasoup-3p-40s-t3medium-wait",
    "loadtest-webrtc-final-mediasoup-3p-40s-t3medium-retry",
]

index_livekit_names = [
    "loadtest-webrtc-final-livekit-2p-t3medium-retry",
    "loadtest-webrtc-final-livekit-8p-t3medium-retry",
    "loadtest-webrtc-final-livekit-3p-10s-t3medium-retry",
    "loadtest-webrtc-final-livekit-3p-40s-t3medium-retry",
]

index_list_names = index_kurento_names + index_mediasoup_names + index_livekit_names

node_types = ["browseremulator", "masternode", "medianode"]
packet_types = ["inbound", "outbound"]

seconds_per_fragment = 17

start_end_times = pd.read_json("dfs_final/start-end-times.json", orient="index")
start_end_times["from"] = pd.to_datetime(
    start_end_times["from"], format="ISO8601", utc=True
).dt.tz_convert("UTC")
start_end_times["to"] = pd.to_datetime(
    start_end_times["to"], format="ISO8601", utc=True
).dt.tz_convert("UTC")


def timestamp_to_secs(df_node, index, cpu_times=True):
    df_tmp = df_node.copy()
    df_tmp["@timestamp"] = pd.to_datetime(df_tmp["@timestamp"], format="ISO8601")
    # tmp_serie = pd.Series(
    #     [df_tmp["@timestamp"].max(), start_end_times.loc[index, "to"]]
    # )
    #end_time = tmp_serie.min() if cpu_times else tmp_serie.max()
    end_time = start_end_times.loc[index, "to"]
    df_tmp = df_tmp[df_tmp["@timestamp"] < end_time]
    # tmp_serie = pd.Series(
    #     [df_tmp["@timestamp"].min(), start_end_times.loc[index, "from"]]
    # )
    # start_time = tmp_serie.max() if cpu_times else tmp_serie.min()
    start_time = start_end_times.loc[index, "from"]
    df_tmp["timestamp_secs"] = (df_tmp["@timestamp"] - start_time).dt.total_seconds()
    df_tmp = df_tmp[df_tmp["timestamp_secs"] >= 0]
    return df_tmp

In [22]:
typologies = ["2p"]

for typology in typologies:
    lk_target_names = filter(
            lambda x: typology in x and "livekit" in x and "retry" in x, index_list_names
        )
    kur_target_names = filter(
            lambda x: typology in x and "kurento" in x and "retry" in x, index_list_names
        )
    df_outbound = pd.DataFrame(columns=["kurento", "livekit"])
    df_inbound = pd.DataFrame(columns=["kurento", "livekit"])

    for target_name in kur_target_names:
        outbound = pd.read_csv(f"dfs_final/{target_name}-webrtc-stats-outbound.csv")
        outbound = outbound.drop(columns=["@timestamp", "user_id", "session_id"])
        inbound = pd.read_csv(f"dfs_final/{target_name}-webrtc-stats-inbound.csv")
        inbound = inbound.drop(columns=["@timestamp", "user_id", "session_id"])
        print("kurento outbound data:")
        for col in outbound.columns:
            print("- " + col)
        inbound = pd.read_csv(f"dfs_final/{target_name}-webrtc-stats-inbound.csv")
        inbound = inbound.drop(columns=["@timestamp", "user_id", "session_id"])
        print("kurento inbound data:")
        for col in inbound.columns:
            print("- " + col)
        break

    for target_name in lk_target_names:
        outbound = pd.read_csv(f"dfs_final/{target_name}-webrtc-stats-outbound.csv")
        outbound = outbound.drop(columns=["@timestamp", "user_id", "session_id", "audio_type", "audio_streamId", "audio_timestamp"])
        print("livekit outbound data:")
        for col in outbound.columns:
            print("- " + col)
        inbound = pd.read_csv(f"dfs_final/{target_name}-webrtc-stats-inbound.csv")
        inbound = inbound.drop(columns=["@timestamp", "user_id", "session_id", "audio_type", "audio_timestamp"])
        print("livekit inbound data:")
        for col in inbound.columns:
            print("- " + col)
        break

kurento outbound data:
- audio_bytesSent
- audio_packetsSent
- audio_nackCount
- video_bytesSent
- video_packetsSent
- video_framesEncoded
- video_nackCount
- video_firCount
- video_pliCount
- video_qpSum
kurento inbound data:
- audio_bytesReceived
- audio_packetsReceived
- audio_packetsLost
- audio_jitter
- video_bytesReceived
- video_packetsReceived
- video_packetsLost
- video_jitter
- video_framesDecoded
- video_nackCount
- video_firCount
- video_pliCount
livekit outbound data:
- audio_packetsSent
- audio_bytesSent
- audio_nackCount
livekit inbound data:
- audio_bytesReceived
- audio_concealedSamples
- audio_jitter
- audio_silentConcealedSamples
- audio_totalAudioEnergy
- audio_totalSamplesDuration
- audio_fecPacketsDiscarded
- audio_fecPacketsReceived
- audio_insertedSamplesForDeceleration
- audio_jitterBufferDelay
- audio_jitterBufferEmittedCount
- audio_lastPacketReceivedTimestamp
- audio_packetsDiscarded
- audio_packetsLost
- audio_packetsReceived
- audio_removedSamplesForAccele